In [1]:
import numpy as np
import plotly as py
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
bt = pd.read_csv('Data/bt.csv', sep=';')	
sp = pd.read_csv('Data/sp.csv', sep=';')
lu = pd.read_csv('Data/lu.csv', sep=';')

# Data treatment

In [3]:
bt.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,bt-mz.D.8,00:02:25,00:02:24,00:02:25
1,4,12,16,bt-mz.D.12,00:01:52,00:01:52,00:01:52
2,4,16,12,bt-mz.D.16,00:01:51,00:01:50,00:01:50
3,4,24,8,bt-mz.D.24,00:01:39,00:01:39,00:01:39
4,4,32,6,bt-mz.D.32,00:01:46,00:01:45,00:01:45


In [4]:
sp.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,sp-mz.D.8,00:01:29,00:01:30,00:01:30
1,4,12,16,sp-mz.D.12,00:01:22,00:01:20,00:01:22
2,4,16,12,sp-mz.D.16,00:01:06,00:01:07,00:01:08
3,4,24,8,sp-mz.D.24,00:00:59,00:00:57,00:00:57
4,4,32,6,sp-mz.D.32,00:00:58,00:00:58,00:00:59


In [5]:
lu.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,lu-mz.D.8,00:02:31,00:02:30,00:02:32
1,4,12,16,lu-mz.D.12,00:23:33,00:23:40,00:23:25
2,4,16,12,lu-mz.D.16,00:02:36,00:02:37,00:02:38
3,6,12,24,lu-mz.D.12,00:18:19,00:18:27,00:18:37
4,8,16,24,lu-mz.D.16,00:01:17,00:01:16,00:01:16


In [6]:
bt['partition'] = bt['partition'].astype(str)
sp['partition'] = sp['partition'].astype(str)
lu['partition'] = lu['partition'].astype(str)

In [7]:
for n in range(0,3):
    bt[f'elapsed{n+1}'] = bt[f'elapsed{n+1}'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))
    sp[f'elapsed{n+1}'] = sp[f'elapsed{n+1}'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))
    lu[f'elapsed{n+1}'] = lu[f'elapsed{n+1}'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))


# Mean and standart deviation elapsed time

In [11]:
bt['mean elapsed t'] = bt[['elapsed1', 'elapsed2', 'elapsed3']].mean(axis=1)
sp['mean elapsed t'] = sp[['elapsed1', 'elapsed2', 'elapsed3']].mean(axis=1)
lu['mean elapsed t'] = lu[['elapsed1', 'elapsed2', 'elapsed3']].mean(axis=1)
bt['sd elapsed t'] = bt[['elapsed1', 'elapsed2', 'elapsed3']].std(axis=1)
sp['sd elapsed t'] = sp[['elapsed1', 'elapsed2', 'elapsed3']].std(axis=1)
lu['sd elapsed t'] = lu[['elapsed1', 'elapsed2', 'elapsed3']].std(axis=1)

In [12]:
bt.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t
0,4,8,24,bt-mz.D.8,145,144,145,144.666667,0.57735
1,4,12,16,bt-mz.D.12,112,112,112,112.000000,0.00000
2,4,16,12,bt-mz.D.16,111,110,110,110.333333,0.57735
3,4,24,8,bt-mz.D.24,99,99,99,99.000000,0.00000
4,4,32,6,bt-mz.D.32,106,105,105,105.333333,0.57735


In [13]:
sp.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t
0,4,8,24,sp-mz.D.8,89,90,90,89.666667,0.577350
1,4,12,16,sp-mz.D.12,82,80,82,81.333333,1.154701
2,4,16,12,sp-mz.D.16,66,67,68,67.000000,1.000000
3,4,24,8,sp-mz.D.24,59,57,57,57.666667,1.154701
4,4,32,6,sp-mz.D.32,58,58,59,58.333333,0.577350


In [14]:
lu.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t
0,4,8,24,lu-mz.D.8,151,150,152,151.000000,1.000000
1,4,12,16,lu-mz.D.12,1413,1420,1405,1412.666667,7.505553
2,4,16,12,lu-mz.D.16,156,157,158,157.000000,1.000000
3,6,12,24,lu-mz.D.12,1099,1107,1117,1107.666667,9.018500
4,8,16,24,lu-mz.D.16,77,76,76,76.333333,0.577350


# Theoretic speedup data

In [15]:
def get_speed_up(mode):
    it = []; speed_up = []
    aux = 3
    for node in [4,6,8]:
        header = True
        file = f'./Out/{mode}-{node}-1.out'
        with open(file, 'r') as f:
            found = False
            for exe in range(8) if mode == 'bt' or mode == 'sp' else range(aux):
                # ignore the first 6 lines
                for i in range(6) if header is True else range(3):
                    f.readline()
                header = False
                # read the next line
                line = f.readline()
                # get the number after the word 'Iterations:'
                if not found:
                    l = 2 if mode == 'bt' or mode == 'lu' else 7
                    it_exe = int(line.split(' ')[l])
                    found = True
                it.append(it_exe)
                # ignore the next 5 lines
                for i in range(5):
                    f.readline()
                # read the next line
                line = f.readline()
                # get the last number in the line
                speed_up_exe = float(line.split(' ')[-1])
                speed_up.append(speed_up_exe)
                # ignore the next 75 lines
                for i in range(63):
                    f.readline()
                if mode == 'sp':
                    for i in range(13):
                        f.readline()
                elif mode == 'lu':
                    for i in range(6):
                        f.readline()
        print(f'end mode {mode} with {node} nodes')
        aux = 1
    return it, speed_up

In [16]:
# get the speed up for each mode
it_bt, speed_up_bt = get_speed_up('bt')
it_sp, speed_up_sp = get_speed_up('sp')
it_lu, speed_up_lu = get_speed_up('lu')

end mode bt with 4 nodes
end mode bt with 6 nodes
end mode bt with 8 nodes
end mode sp with 4 nodes
end mode sp with 6 nodes
end mode sp with 8 nodes
end mode lu with 4 nodes
end mode lu with 6 nodes
end mode lu with 8 nodes


In [17]:
bt['theoretical speedup'] = speed_up_bt
sp['theoretical speedup'] = speed_up_sp
lu['theoretical speedup'] = speed_up_lu
bt['iterations'] = it_bt
sp['iterations'] = it_sp
lu['iterations'] = it_lu

In [18]:
bt.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations
0,4,8,24,bt-mz.D.8,145,144,145,144.666667,0.57735,191.98,250
1,4,12,16,bt-mz.D.12,112,112,112,112.000000,0.00000,191.71,250
2,4,16,12,bt-mz.D.16,111,110,110,110.333333,0.57735,191.96,250
3,4,24,8,bt-mz.D.24,99,99,99,99.000000,0.00000,191.63,250
4,4,32,6,bt-mz.D.32,106,105,105,105.333333,0.57735,191.88,250


In [19]:
sp.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations
0,4,8,24,sp-mz.D.8,89,90,90,89.666667,0.577350,192.00,500
1,4,12,16,sp-mz.D.12,82,80,82,81.333333,1.154701,190.51,500
2,4,16,12,sp-mz.D.16,66,67,68,67.000000,1.000000,192.00,500
3,4,24,8,sp-mz.D.24,59,57,57,57.666667,1.154701,190.51,500
4,4,32,6,sp-mz.D.32,58,58,59,58.333333,0.577350,192.00,500


In [20]:
lu.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations
0,4,8,24,lu-mz.D.8,151,150,152,151.000000,1.000000,192.0,300
1,4,12,16,lu-mz.D.12,1413,1420,1405,1412.666667,7.505553,192.0,300
2,4,16,12,lu-mz.D.16,156,157,158,157.000000,1.000000,192.0,300
3,6,12,24,lu-mz.D.12,1099,1107,1117,1107.666667,9.018500,288.0,300
4,8,16,24,lu-mz.D.16,77,76,76,76.333333,0.577350,384.0,300


# Seedup and efficiency data

In [21]:
# sequential time for each mode
t1_bt = 2206
t1_sp = 1151.58
t1_lu = 1818

In [22]:
bt['empirical speedup'] = t1_bt/bt['mean elapsed t']
sp['empirical speedup'] = t1_sp/sp['mean elapsed t']
lu['empirical speedup'] = t1_lu/lu['mean elapsed t']
bt['efficiency'] = bt['empirical speedup']*8/(bt['cpus-per-task']* bt['ntasks'])
sp['efficiency'] = sp['empirical speedup']*8/(sp['cpus-per-task']* sp['ntasks'])
lu['efficiency'] = lu['empirical speedup']*8/(lu['cpus-per-task']* lu['ntasks'])

In [61]:
bt.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations,empirical speedup,efficiency
0,4,8,24,bt-mz.D.8,145,144,145,144.666667,0.57735,191.98,250,15.248848,0.635369
1,4,12,16,bt-mz.D.12,112,112,112,112.000000,0.00000,191.71,250,19.696429,0.820685
2,4,16,12,bt-mz.D.16,111,110,110,110.333333,0.57735,191.96,250,19.993958,0.833082
3,4,24,8,bt-mz.D.24,99,99,99,99.000000,0.00000,191.63,250,22.282828,0.928451
4,4,32,6,bt-mz.D.32,106,105,105,105.333333,0.57735,191.88,250,20.943038,0.872627


In [62]:
sp.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations,empirical speedup,efficiency
0,4,8,24,sp-mz.D.8,89,90,90,89.666667,0.577350,192.00,500,12.842900,0.535121
1,4,12,16,sp-mz.D.12,82,80,82,81.333333,1.154701,190.51,500,14.158770,0.589949
2,4,16,12,sp-mz.D.16,66,67,68,67.000000,1.000000,192.00,500,17.187761,0.716157
3,4,24,8,sp-mz.D.24,59,57,57,57.666667,1.154701,190.51,500,19.969595,0.832066
4,4,32,6,sp-mz.D.32,58,58,59,58.333333,0.577350,192.00,500,19.741371,0.822557


In [63]:
lu.head()

,nodes,ntasks,cpus-per-task,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t,theoretical speedup,iterations,empirical speedup,efficiency
0,4,8,24,lu-mz.D.8,151,150,152,151.000000,1.000000,192.0,300,12.039735,0.501656
1,4,12,16,lu-mz.D.12,1413,1420,1405,1412.666667,7.505553,192.0,300,1.286928,0.053622
2,4,16,12,lu-mz.D.16,156,157,158,157.000000,1.000000,192.0,300,11.579618,0.482484
3,6,12,24,lu-mz.D.12,1099,1107,1117,1107.666667,9.018500,288.0,300,1.641288,0.045591
4,8,16,24,lu-mz.D.16,77,76,76,76.333333,0.577350,384.0,300,23.816594,0.496179


# bt application analysis

## Elapsed time vs ntasks (MPIs)

In [30]:
fig = px.line(x=bt['ntasks'], y=bt['mean elapsed t'],
                markers=True,
                color=bt['nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.update_layout(title='Mean elapsed time for bt application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Elapsed ime (s)')

fig.show()

## Speedup vs ntasks (MPIs)

In [31]:
fig = px.line(x=bt['ntasks'], y=bt['empirical speedup'],
                markers=True,
                color=bt['nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.update_layout(title='Mean speedup for bt application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Speedup')

fig.show()

## Efficiency vs ntasks (MPIs)

In [32]:
fig = px.line(x=bt['ntasks'], y=bt['efficiency'],
                markers=True,
                color=bt['nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.update_layout(title='Mean efficiency for bt application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Efficiency')

fig.show()

# sp application analysis

## Elapsed time vs ntasks (MPIs)

In [33]:
fig = px.line(x=sp['ntasks'], y=sp['mean elapsed t'],
                markers=True,
                color=sp['nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.update_layout(title='Mean elapsed time for sp application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Elapsed ime (s)')

fig.show()

## Speedup vs ntasks (MPIs)

In [34]:
fig = px.line(x=sp['ntasks'], y=sp['empirical speedup'],
                markers=True,
                color=sp['nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.update_layout(title='Mean speedup for sp application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Speedup')

                    

fig.show()

## Efficiency vs ntasks (MPIs)

In [35]:
fig = px.line(x=sp['ntasks'], y=sp['efficiency'],
                markers=True,
                color=sp['nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.update_layout(title='Mean efficiency for sp application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Efficiency')
fig.show()

# lu application analysis

## Elapsed time vs ntasks (MPIs)

In [36]:
fig = px.line(x=lu['ntasks'], y=lu['mean elapsed t'],
                markers=True,
                color=lu['nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.update_layout(title='Mean elapsed time for lu application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Elapsed ime (s)')

fig.show()

## Speedup vs ntasks (MPIs)

In [37]:
fig = px.line(x=lu['ntasks'], y=lu['empirical speedup'],
                markers=True,
                color=lu['nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.update_layout(title='Mean speedup for lu application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Speedup')
fig.show()

## Efficiency vs ntasks (MPIs)

In [38]:
fig = px.line(x=lu['ntasks'], y=lu['efficiency'],
                markers=True,
                color=lu['nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.update_layout(title='Mean efficiency for lu application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Efficiency')
fig.show()

# Economic analysis

In [39]:
cost = pd.read_csv('Data/cost.csv', sep='|')
cost.head()

,Nodes,JobID,CPUTime,NCPUS,ReqCPUS,ConsumedEnergy,AveCPUFreq,JobName,Elapsed
0,4,28076954.0,07:44:00,192,192,170.18K,1.23M,bt-mz.D.8,00:02:25
1,4,28077461.0,05:20:00,192,192,144.33K,277.44M,bt-mz.D.192,00:01:40
2,8,28024862.0,07:53:36,384,384,171.94K,28.29M,bt-mz.D.16,00:01:14
3,8,28024867.0,07:21:36,384,384,182.28K,5.26G,bt-mz.D.384,00:01:09
4,4,28024930.0,04:48:00,192,192,112.23K,900K,sp-mz.D.8,00:01:30


In [40]:
cost['JobName'] = cost['JobName'].astype(str)

In [41]:
cost['ntasks'] = cost['JobName'].str.split('.').str[-1]
cost['ntasks'] = cost['ntasks'].astype(int)

In [43]:
cost['Elapsed'] = cost['Elapsed'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))

In [44]:
cost['ConsumedEnergy'] = cost['ConsumedEnergy'].apply(lambda x: float(x[:-1]) if x[-1] == 'K' else float(x))

In [46]:
cost.head()

,Nodes,JobID,CPUTime,NCPUS,ReqCPUS,ConsumedEnergy,AveCPUFreq,JobName,Elapsed,ntasks
0,4,28076954.0,07:44:00,192,192,170.18,1.23M,bt-mz.D.8,145,8
1,4,28077461.0,05:20:00,192,192,144.33,277.44M,bt-mz.D.192,100,192
2,8,28024862.0,07:53:36,384,384,171.94,28.29M,bt-mz.D.16,74,16
3,8,28024867.0,07:21:36,384,384,182.28,5.26G,bt-mz.D.384,69,384
4,4,28024930.0,04:48:00,192,192,112.23,900K,sp-mz.D.8,90,8


In [47]:
cost['Mean power'] = cost['ConsumedEnergy']/cost['Elapsed']

In [48]:
cost.head()

,Nodes,JobID,CPUTime,NCPUS,ReqCPUS,ConsumedEnergy,AveCPUFreq,JobName,Elapsed,ntasks,Mean power
0,4,28076954.0,07:44:00,192,192,170.18,1.23M,bt-mz.D.8,145,8,1.173655
1,4,28077461.0,05:20:00,192,192,144.33,277.44M,bt-mz.D.192,100,192,1.443300
2,8,28024862.0,07:53:36,384,384,171.94,28.29M,bt-mz.D.16,74,16,2.323514
3,8,28024867.0,07:21:36,384,384,182.28,5.26G,bt-mz.D.384,69,384,2.641739
4,4,28024930.0,04:48:00,192,192,112.23,900K,sp-mz.D.8,90,8,1.247000


In [49]:
cost['Cost'] = (cost['Mean power']*cost['Elapsed']*0.2)/3600

## bt application

In [51]:
bt_cost = cost[cost['JobName'].str.split('.').str[0] == 'bt-mz']

fig = px.line(x=bt_cost['ntasks'], y=bt_cost['Mean power'],
                markers=True,
                color=bt_cost['Nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])
fig.update_layout(title='Mean power for bt application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Mean power (kW)')

fig.show()

In [52]:
fig = px.line(x=bt_cost['ntasks'], y=bt_cost['Cost'],
                markers=True,
                color=bt_cost['Nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])

fig.update_layout(title='Mean cost for bt application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Cost (€)')

fig.show()

## sp application

In [53]:
sp_cost = cost[cost['JobName'].str.split('.').str[0] == 'sp-mz']

fig = px.line(x=sp_cost['ntasks'], y=sp_cost['Mean power'],
                markers=True,
                color=sp_cost['Nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])

fig.update_layout(title='Mean power for sp application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Mean power (kW)')

fig.show()

In [54]:
fig = px.line(x=sp_cost['ntasks'], y=sp_cost['Cost'],
                markers=True,
                color=sp_cost['Nodes'],
                color_discrete_sequence=['#440154', '#21918c', '#fde725'])

fig.update_layout(title='Mean cost for sp application',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Cost (€)')

fig.show()

In [55]:
total_cost = cost['Cost'].sum()
print('Total cost: ', total_cost, '€')

total_cost_bt = bt_cost['Cost'].sum()
print('Total cost for bt application: ', total_cost_bt, '€')

total_cost_sp = sp_cost['Cost'].sum()
print('Total cost for sp application: ', total_cost_sp, '€')

Total cost:  0.06376000000000001 €
Total cost for bt application:  0.037151666666666666 €
Total cost for sp application:  0.026608333333333338 €


# Schedule lu application analysis

In [56]:
df_s = pd.read_csv('Data/schedule.csv', sep='|')
df_s['duration'] = df_s['duration'].astype(str)
df_s['nodes'] = df_s['nodes'].astype(str)
df_s

,nodes,schedule,ntasks,id,duration,status,jobname
0,4,static,8,28025472.0,00:06:43,COMPLETED,lu-mz.D.8
1,4,static,12,28025472.1,00:43:08,COMPLETED,lu-mz.D.12
2,4,static,16,28025472.2,00:03:35,COMPLETED,lu-mz.D.16
3,6,static,12,28025472.3,00:51:00,COMPLETED,lu-mz.D.12
4,8,static,16,28025472.4,00:03:23,COMPLETED,lu-mz.D.16
5,4,dynamic,8,28137084.0,00:13:59,COMPLETED,lu-mz.D.8
6,4,dynamic,12,28137084.1,00:46:30,COMPLETED,lu-mz.D.12
7,4,dynamic,16,28137084.2,00:08:05,COMPLETED,lu-mz.D.16
8,6,dynamic,12,28138208.0,00:56:17,COMPLETED,lu-mz.D.12
9,8,dynamic,16,28138208.1,00:07:01,COMPLETED,lu-mz.D.16


In [57]:
df_s['duration'] = df_s['duration'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))
df_s

,nodes,schedule,ntasks,id,duration,status,jobname
0,4,static,8,28025472.0,403,COMPLETED,lu-mz.D.8
1,4,static,12,28025472.1,2588,COMPLETED,lu-mz.D.12
2,4,static,16,28025472.2,215,COMPLETED,lu-mz.D.16
3,6,static,12,28025472.3,3060,COMPLETED,lu-mz.D.12
4,8,static,16,28025472.4,203,COMPLETED,lu-mz.D.16
5,4,dynamic,8,28137084.0,839,COMPLETED,lu-mz.D.8
6,4,dynamic,12,28137084.1,2790,COMPLETED,lu-mz.D.12
7,4,dynamic,16,28137084.2,485,COMPLETED,lu-mz.D.16
8,6,dynamic,12,28138208.0,3377,COMPLETED,lu-mz.D.12
9,8,dynamic,16,28138208.1,421,COMPLETED,lu-mz.D.16


## Static

In [58]:
fig = px.bar(df_s[df_s['schedule'] == 'static'], x='ntasks', y='duration', color='nodes', barmode='group',
            color_discrete_sequence=["#440154", "#21918c", "#fde725"],
            text='duration')

fig.update_layout(title='Static schedule',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Duration (s)')
fig.update_yaxes(range=[0, 3500])
fig.show()

## Dynamic

In [59]:
fig = px.bar(df_s[df_s['schedule'] == 'dynamic'], x='ntasks', y='duration', color='nodes', barmode='group',
            color_discrete_sequence=["#440154", "#21918c", "#fde725"],
            text='duration')

fig.update_layout(title='Dynamic schedule',
                    legend_title_text='Nodes',
                    xaxis_title='Number of tasks (MPIs)',
                    yaxis_title='Duration (s)')
fig.update_yaxes(range=[0, 3500])

fig.show()

In [60]:
diff =  df_s[df_s['schedule'] == 'dynamic']['duration'].values - df_s[df_s['schedule'] == 'static']['duration'].values

diff

array([436, 202, 270, 317, 218], dtype=int64)